## Расчет метрик
Для сайта, продающего билеты в кино, имеется набор данных (логов) за период с сентября по ноябрь.


В примере показана таблица данных с дневной агрегацией. Она содержит:
 - Дату.
 - Идентификатор сессии пользователя.
 - Идентификатор пользователя.
 - Ярлык события
 - Доход от этих событий на определённую дату у определённого пользователя за одну сессию.




Используйте данные, чтобы рассчитать метрики отдельно за сентябрь, октябрь и ноябрь:
1. Количество сессий (визитов).
2. MAU.
3. DAU.
4. Revenue.
5. ARPU.
6. ARPPU.
7. Session / user.
8. Количество целевых действий (целевое действие — событие, за которое получен доход).


In [29]:
import numpy as np
import pandas as pd

In [5]:
df_logs = pd.read_excel('dataset-1.xlsx', parse_dates=['Дата'])
df_logs.head()

,Дата,Session ID,Client ID,Ярлык события,Доход
0,2018-09-02,6517411217615201018blqo010gfv,42325189,Search Field,0.0
1,2018-09-02,6517439917954201018jogp8guf,49083805,Movie cover,0.0
2,2018-09-03,6517492978275201018xnvoyeqm,27598678,Event Place,0.0
3,2018-09-03,651743624394010201018cg2zi5ol8z276c3x16y9zfr,5760746,Movie cover,0.0
4,2018-09-03,6517492978275201018xnvoyeqm,27598678,Main - Menu,0.0


In [6]:
df_logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20342 entries, 0 to 20341
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Дата           20342 non-null  datetime64[ns]
 1   Session ID     20342 non-null  object        
 2   Client ID      20342 non-null  int64         
 3   Ярлык события  20342 non-null  object        
 4   Доход          20342 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 794.7+ KB


In [16]:
df_logs['Месяц'] = df_logs['Дата'].dt.month
df_logs.head()

,Дата,Session ID,Client ID,Ярлык события,Доход,Месяц
0,2018-09-02,6517411217615201018blqo010gfv,42325189,Search Field,0.0,9
1,2018-09-02,6517439917954201018jogp8guf,49083805,Movie cover,0.0,9
2,2018-09-03,6517492978275201018xnvoyeqm,27598678,Event Place,0.0,9
3,2018-09-03,651743624394010201018cg2zi5ol8z276c3x16y9zfr,5760746,Movie cover,0.0,9
4,2018-09-03,6517492978275201018xnvoyeqm,27598678,Main - Menu,0.0,9


### 1. Количество сессий (визитов)

In [81]:
sessions_count = df_logs.groupby('Месяц').agg(session_count = ('Session ID', 'nunique')) 
df_logs['Session ID'].nunique()

16907

### 2. MAU

In [20]:
mou = df_logs.groupby('Месяц').agg(users_count = ('Client ID', 'nunique'))
mou

,users_count
Месяц,
9,2511
10,8520
11,3647


### 3. DAU

In [95]:
dau = df_logs.groupby('Дата').agg(user_count = ('Client ID', 'nunique'))
dau.head(20)

,user_count
Дата,
2018-09-02,2
2018-09-03,2
2018-09-05,1
2018-09-06,2
2018-09-07,1
2018-09-08,2
2018-09-09,1
2018-09-10,2
2018-09-11,2


### 4. Revenue

In [28]:
revenue = df_logs.groupby('Месяц').agg(revenue = ('Доход', sum))
revenue

,revenue
Месяц,
9,2.779013e+07
10,2.986842e+07
11,1.960508e+07


### 5. ARPU

In [72]:
users_count = df_logs.groupby('Месяц').agg(count_users = ('Client ID', 'nunique'), revenue = ('Доход', sum))
users_count['arpu'] = users_count['revenue'] / users_count['count_users']
users_count

,count_users,revenue,arpu
Месяц,,,
9,2511,2.779013e+07,11067.353869
10,8520,2.986842e+07,3505.682983
11,3647,1.960508e+07,5375.674394


### 6. ARPPU

In [49]:
users_count = df_logs[df_logs['Доход'] > 0]\
    .groupby('Месяц')\
    .agg(count_users = ('Client ID', 'nunique'), revenue = ('Доход', sum))

users_count['arppu'] = users_count['revenue'] / users_count['count_users']

,count_users,revenue,arppu
Месяц,,,
9,1268,2.779013e+07,21916.502811
10,1195,2.986842e+07,24994.492898
11,890,1.960508e+07,22028.184848


### 7. Session / user

In [101]:
session_by_user = df_logs.groupby('Месяц')\
                    .agg(users = ('Client ID', 'nunique'),\
                         sessions = ('Session ID', 'count'))
session_by_user['share'] = session_by_user['sessions'] / session_by_user['users']

session_by_user

,users,sessions,share
Месяц,,,
9,2511,3309,1.317802
10,8520,12067,1.416315
11,3647,4966,1.361667


### 8. Количество целевых действий

In [104]:
target_action = df_logs[df_logs['Доход'] > 0].groupby(['Месяц', 'Ярлык события'])\
        .agg(count = ('Session ID', 'count'))
target_action

count
Месяц Ярлык события                   
9     Desktop - To Payment Card    421
      Mobile - To Payment Card     580
      Place Select                 550
      Promocode                     68
      Search Field                   1
10    Desktop - To Payment Card    457
      Mobile - To Payment Card     601
      Movie cover                    1
      Place Select                 413
      Promocode                     46
11    Desktop - To Payment Card    317
      Mobile - To Payment Card     471
      Movie cover                    1
      Place Select                 314
      Promocode                     30